# DES Y6 GOLD re-import

- The raw data for DES Y6 GOLD has disappeared.
- Looks like it was imported originally with a threshold of 1_000_000. but this is resulting in some very large partitions.
- Trying this time with 750_000

In [ ]:
from dask.distributed import Client
from hats_import.pipeline import pipeline_with_client, pipeline
from hats_import import ImportArguments, CollectionArguments, VerificationArguments
import hats_import

hats_import.__version__

In [ ]:
args = ImportArguments.reimport_from_hats(
    "/epyc/data3/hats/catalogs/des/des_y6_gold",
    "/epyc/data3/hats/catalogs/des/des_y6_gold_collection/",
    output_artifact_name="des_y6_gold",
    
    pixel_threshold=750_000,
    # sort_columns="QUICK_OBJECT_ID",
    highest_healpix_order=6,
    skymap_alt_orders=[2, 4],
    row_group_kwargs={"num_rows": 100_000},

    completion_email_address="delucchi@andrew.cmu.edu",
    progress_bar=True,
    simple_progress_bar=True,
)

with Client(local_directory="/epyc/data3/hats/tmp/", n_workers=10, threads_per_worker=1) as client:
    pipeline_with_client(args, client)

In [ ]:
args = (
    CollectionArguments(
        output_artifact_name="des_y6_gold_collection",
        output_path="/epyc/data3/hats/catalogs/des/",
        completion_email_address="delucchi@andrew.cmu.edu",
        progress_bar=True,
        simple_progress_bar=True,
    )
    .catalog(
        output_artifact_name="des_y6_gold",
    )
    .add_margin(margin_threshold=5.0, is_default=True)
)

with Client(local_directory="/epyc/data3/hats/tmp/", n_workers=10, threads_per_worker=1) as client:
    pipeline_with_client(args, client)

In [ ]:
args = VerificationArguments(
    input_catalog_path="/epyc/data3/hats/catalogs/des/des_y6_gold_collection",
    output_path="./verification/des_y6_gold",
)

pipeline(args)